# Сводная таблица pivot_table

Сводная таблица - это мощный инструмент для обобщения и представления данных.

В Pandas для подготовки сводных таблиц вызывают метод **pivot_table()**.

Аргументы метода:

• index — ключи для группировки по индексу (строке)

• columns — ключи для группировки столбцов

• values — значения, по которым мы хотим увидеть сводную таблицу (столбцы для агрегирования)

• aggfunc — функция, применяемая к значениям (т.е. что делать с повторяющимися значениями)

<img src="https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/pic/pivot_table_pandas.png" >



Построим сводную таблицу для задачи по SEO-оптимизации.
Для начала загрузите дата-сет data.final, полученный в результате выполнения предыдущей задачи (объединения таблиц), сохраните ешл в переменной df.

In [1]:
import pandas as pd
df = pd.read_csv('data_final.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   subcategory_id    1928 non-null   object
 1   source            1928 non-null   object
 2   visits            1928 non-null   int64 
 3   subcategory_name  1928 non-null   object
 4   category_id       1928 non-null   object
 5   category_name     1928 non-null   object
dtypes: int64(1), object(5)
memory usage: 90.5+ KB


## Поворот данных

Создавать сводную таблицу **(pivot_table)** лучше всего последовательно. Добавляйте элементы по одному и проверяйте каждый шаг, чтобы убедиться, что вы получаете ожидаемые результаты. Самая простая сводная таблица должна иметь **DataFrame** и индекс **(index)**. 

В этом примере давайте использовать **category_name** в качестве индекса:

**pd.pivot_table(df, index=["category_name"])**

In [2]:
data_pivot = df.pivot_table(index=["category_name"])
data_pivot

TypeError: agg function failed [how->mean,dtype->object]

У вас может быть несколько индексов. Фактически, большинство аргументов **pivot_table** могут принимать несколько значений в качестве элементов списка. Добавьте в список индексов переменную **subcategory_name**.

In [3]:
data_pivot = df.pivot_table(index=['category_name', 'subcategory_name'])
data_pivot

TypeError: agg function failed [how->mean,dtype->object]

Вы могли заметить, что сводная таблица достаточно умна, чтобы начать агрегирование данных и их обобщение, группируя категории товаров (**category_name**) с их подкатегориями (**subcategory_name**). Теперь мы начинаем понимать, что может сделать для нас сводная таблица.

Поскольку в данном случае столбец с количественной переменной только один (**visits**), он автоматически определился в качестве значения параметра **values**. Однако, можно явно определить столбцы, которые нам нужны, с помощью этого параметра **values**:

**pd.pivot_table(df, index=['category_name', 'subcategory_name'], values=["visits"])**

In [4]:
data_pivot = df.pivot_table(index=['category_name', 'subcategory_name'],
                                    values='visits')
data_pivot


visits
category_name subcategory_name                  
Авто          Автоакустика               10674.0
              Автомагнитолы              13236.5
              Автомобильные инверторы      147.5
              Автомобильные компрессоры    598.5
              Автомобильные телевизоры    1610.5
...                                          ...
Электроника   Чехлы                       8170.0
              Чехлы-аккумуляторы           275.0
              Штативы и моноподы          1846.0
              Экшн-камеры                 8566.5
              Электронные книги           7939.5

[934 rows x 1 columns]

Столбец цен (**visits**) по умолчанию усредняет данные, но мы можем произвести подсчет количества или суммы. Добавить их можно с помощью параметра **aggfunc**:

**pd.pivot_table(df, index=['category_name', 'subcategory_name'], values=["visits"], aggfunc='sum')**

In [6]:
data_pivot = df.pivot_table(index=['category_name', 'subcategory_name'],
                                    values='visits', aggfunc='sum')
data_pivot

visits
category_name subcategory_name                 
Авто          Автоакустика                21348
              Автомагнитолы               26473
              Автомобильные инверторы       295
              Автомобильные компрессоры    1197
              Автомобильные телевизоры     3221
...                                         ...
Электроника   Чехлы                       16340
              Чехлы-аккумуляторы            550
              Штативы и моноподы           3692
              Экшн-камеры                 17133
              Электронные книги           15879

[934 rows x 1 columns]

**Aggfunc** может принимать список функций.

Давайте попробуем узнать среднее значение и количество:



In [7]:
data_pivot = df.pivot_table(index=['category_name', 'subcategory_name'],
                                    values='visits', aggfunc=['mean', 'count'])
data_pivot

mean  count
                                          visits visits
category_name subcategory_name                         
Авто          Автоакустика               10674.0      2
              Автомагнитолы              13236.5      2
              Автомобильные инверторы      147.5      2
              Автомобильные компрессоры    598.5      2
              Автомобильные телевизоры    1610.5      2
...                                          ...    ...
Электроника   Чехлы                       8170.0      2
              Чехлы-аккумуляторы           275.0      2
              Штативы и моноподы          1846.0      2
              Экшн-камеры                 8566.5      2
              Электронные книги           7939.5      2

[934 rows x 2 columns]

Если мы хотим увидеть визиты с разбивкой по источникам трафика (**source**), переменная **columns** позволяет нам определить один или несколько столбцов.

Одна из сложностей **pivot_table** - это использование столбцов (**columns**) и значений (**values**).

Помните, что столбцы необязательны - они предоставляют дополнительный способ сегментировать актуальные значения, которые вам нужны.

Функции агрегирования применяются к перечисленным значениям (**values**):

**pd.pivot_table(df, index=['category_name', 'subcategory_name'], values=["visits"], columns=["source"], aggfunc='sum'])**


In [8]:
data_pivot = df.pivot_table(index=['category_name', 'subcategory_name'], values=["visits"], columns=["source"], aggfunc='sum')
data_pivot

visits        
source                                  direct organic
category_name subcategory_name                        
Авто          Автоакустика                5915   15433
              Автомагнитолы               7783   18690
              Автомобильные инверторы      145     150
              Автомобильные компрессоры    407     790
              Автомобильные телевизоры     975    2246
...                                        ...     ...
Электроника   Чехлы                       4582   11758
              Чехлы-аккумуляторы           219     331
              Штативы и моноподы          1152    2540
              Экшн-камеры                 4757   12376
              Электронные книги           4515   11364

[934 rows x 2 columns]

Теперь вы видите название категории и её подкатегории со значениями объёма трафика по каждой.

Как видно, основная категория включает в себя подкатегорию и это представлено в структуре датафрейма: категория отображена иерархически главной над подкатегорией. Такие датафреймы содержат в себе мультииндекс. Часто при работе с такими датафреймами мультииндекс убирают, чтобы категория была отображена на каждой строчке датафрейма:

**pd.pivot_table(df, index=['category_name', 'subcategory_name'], values=["visits"], 
               columns=["source"], aggfunc='sum').reset_index()**


In [9]:
data_pivot = df.pivot_table(index=['category_name', 'subcategory_name'], values=["visits"], 
               columns=["source"], aggfunc='sum').reset_index()
data_pivot


category_name           subcategory_name visits        
source                                          direct organic
0               Авто               Автоакустика   5915   15433
1               Авто              Автомагнитолы   7783   18690
2               Авто    Автомобильные инверторы    145     150
3               Авто  Автомобильные компрессоры    407     790
4               Авто   Автомобильные телевизоры    975    2246
..               ...                        ...    ...     ...
929      Электроника                      Чехлы   4582   11758
930      Электроника         Чехлы-аккумуляторы    219     331
931      Электроника         Штативы и моноподы   1152    2540
932      Электроника                Экшн-камеры   4757   12376
933      Электроника          Электронные книги   4515   11364

[934 rows x 4 columns]

В таблице выше суммы визитов **visits** по каждому из источников **source (direct и organic)** представлены в отдельных столбцах. Такой вид таблицы называется «широкий». Он удобен, когда нужно сравнить значения столбцов. Хорошо заметно, например, что в подкатегории «Автомобильные инверторы» количество визитов из источников **direct** и **organic** практически одинаково, а в подкатегории «Автомагнитолы» из источника **organic** пришло почти в два раза больше посетителей, чем из **direct**.

## Расширенная фильтрация сводной таблицы

После того, как вы сгенерировали свои данные, они находятся в **DataFrame**, поэтому можно фильтровать их, используя обычные методы **DataFrame**.

Если вы хотите посмотреть только на одну категорию, например, "Товары для дома":

**data_pivot = pd.pivot_table(df, index=['category_name', 'subcategory_name'], values='visits', columns='source',
                            aggfunc='sum').reset_index()**
**print((data_pivot[data_pivot['category_name']=='Товары для дома']).head(10))**

In [10]:
data_pivot = pd.pivot_table(df, index=['category_name', 'subcategory_name'], values='visits', columns='source',
                            aggfunc='sum').reset_index()
print((data_pivot[data_pivot['category_name']=='Товары для дома']).head(10))

source    category_name          subcategory_name  direct  organic
646     Товары для дома                Аксессуары     219      329
647     Товары для дома    Аксессуары для готовки     216      326
648     Товары для дома        Банки для хранения     113       58
649     Товары для дома         Блюда и салатники     300      550
650     Товары для дома          Бокалы и стаканы     750     1595
651     Товары для дома                       Бра     265      452
652     Товары для дома                      Вазы     142      142
653     Товары для дома         Вешалки напольные     304      562
654     Товары для дома  Встраиваемые светильники     306      563
655     Товары для дома       Выпечка и запекание     698     1460


Или, например, мы можем узнать на какие товары запросы с органического трафика в разы больше, чем с прямого. Для этого создадим в таблице **data_pivot** новый столбец **'ratio'** и сохраните в нём значение отношения органического трафика **'organic'** к прямому **'direct'**. 

Отсортируем таблицу по столбцу 'ratio' в порядке убывания. Выведите первые 10 строк. 

**data_pivot['ratio'] = data_pivot['organic'] / data_pivot['direct']**

**data_my = data_pivot.sort_values(by = 'ratio', ascending=False)**

**print(data_my.head(10))**

In [11]:
data_pivot['ratio'] = data_pivot['organic'] / data_pivot['direct']
data_my = data_pivot.sort_values(by = 'ratio', ascending=False)
print(data_my.head(10))

source           category_name                    subcategory_name  direct  \
45                        Авто                                Шины   39638   
105            Бытовая техника                   Стиральные машины   22064   
468              Спорт и отдых                          Велосипеды   16009   
205             Детские товары                             Коляски   18079   
609     Строительство и ремонт                  Сварочные аппараты    6175   
678            Товары для дома                 Компьютерные кресла    6208   
330       Компьютерная техника                          Клавиатуры    6224   
921                Электроника  Универсальные внешние аккумуляторы    6146   
866                Электроника                           TV-тюнеры    6275   
117            Бытовая техника                        Холодильники   20920   

source  organic     ratio  
45       153859  3.881604  
105       64570  2.926487  
468       46087  2.878818  
205       52005  2.876542  
6

# Задачи для самостоятельного выполнения

Для начала давайте представим, что мы являемся аналитиками в фирме по продаже компьютеров, программного обеспечения к ним, а также оказываем услуги по техническому сопровождению. Нам поставлена задача проанализировать участие компании в различных аукционах. 

1. Прочитайте и сохраните файл "baza.xlsx" с данными аукционов в переменную data_pd.

In [12]:
data_pd = pd.read_excel('baza.xlsx')

In [13]:
data_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Аукцион     17 non-null     int64 
 1   Контрагент  17 non-null     object
 2   Контакт     17 non-null     object
 3   Менеджер    17 non-null     object
 4   Продукт     17 non-null     object
 5   Количество  17 non-null     int64 
 6   Цена        17 non-null     int64 
 7   Статус      17 non-null     object
dtypes: int64(3), object(5)
memory usage: 1.2+ KB


In [16]:
data_pd.columns

Index(['Аукцион', 'Контрагент', 'Контакт', 'Менеджер', 'Продукт', 'Количество',
       'Цена', 'Статус'],
      dtype='object')

2. Cоздайте сводную таблицу сводную таблицу по столбцу Контрагент:

In [30]:
data_pivot1 = data_pd.pivot_table(index=['Контрагент'], values=['Цена', 'Количество'])
data_pivot1

,Количество,Цена
Контрагент,,
Альма,3.000000,24500.0
Ильин и Ко,3.250000,35775.0
Кружка и ложка,2.750000,27000.0
Микрошкин,3.500000,23275.0
Шахты плюс,1.666667,14900.0


3. Создайте сводную таблицу по нескольким индексируемым столбцам  'Контакт', 'Менеджер':


In [34]:
data_pivot = data_pd.pivot_table(index=['Контакт', 'Менеджер'], values=['Цена', 'Количество'])
data_pivot

,,Количество,Цена
Контакт,Менеджер,,
Виктор Юдин,Павел Попов,2.750000,27000.0
Данил Сидоров,Илья Сергеев,1.666667,14900.0
Женя Сидин,Илья Сергеев,3.000000,24500.0
Сергей Ильин,Илья Сергеев,3.250000,35775.0
Сергей Минин,Павел Попов,3.500000,23275.0


4. По умолчанию сводная таблица выводится по всем числовым полям, однако это не всегда удобно, а иногда и лишено смысла, поэтому можно выводить сводные данные только по отдельным столбцам. Выведите только столбец "Стоимость", для этого добавьте параметр values=['Цена']:

In [35]:
data_pivot = data_pd.pivot_table(index=['Контакт', 'Менеджер'], values=['Цена'])
data_pivot

,,Цена
Контакт,Менеджер,
Виктор Юдин,Павел Попов,27000.0
Данил Сидоров,Илья Сергеев,14900.0
Женя Сидин,Илья Сергеев,24500.0
Сергей Ильин,Илья Сергеев,35775.0
Сергей Минин,Павел Попов,23275.0


5. Столбец с ценой по умолчанию выводит среднее значение, однако нам скорее интересна сумма продаж. Добавляем параметр aggfunc='sum':

In [36]:
data_pivot = data_pd.pivot_table(index=['Контакт', 'Менеджер'], values=['Цена'], aggfunc='sum')
data_pivot

,,Цена
Контакт,Менеджер,
Виктор Юдин,Павел Попов,108000
Данил Сидоров,Илья Сергеев,44700
Женя Сидин,Илья Сергеев,49000
Сергей Ильин,Илья Сергеев,143100
Сергей Минин,Павел Попов,93100


6. С помощью параметра columns выведите в столбцы наименование продуктов:

In [37]:
data_pivot = data_pd.pivot_table(index=['Контакт', 'Менеджер'], columns='Продукт', values=['Цена'], aggfunc='sum')
data_pivot

Цена                                    
Продукт                    Компьютер Монитор     Софт Тех. сопровождение
Контакт       Менеджер                                                  
Виктор Юдин   Павел Попов    83900.0  9300.0      NaN            14800.0
Данил Сидоров Илья Сергеев   38000.0     NaN   6700.0                NaN
Женя Сидин    Илья Сергеев   42000.0     NaN      NaN             7000.0
Сергей Ильин  Илья Сергеев   84300.0     NaN  37600.0            21200.0
Сергей Минин  Павел Попов    43900.0     NaN  46900.0             2300.0

7. Наверное вы обратили внимание, что в ячейках, где нет данных пусто, хотя нам привычнее, что бы в таких полях указывалось бы значение 0. Добавьте параметр fill_value=0:

In [38]:
data_pivot = data_pd.pivot_table(index=['Контакт', 'Менеджер'], columns='Продукт', values=['Цена'],
                                  aggfunc='sum', fill_value=0)
data_pivot

Цена                                  
Продукт                    Компьютер Монитор   Софт Тех. сопровождение
Контакт       Менеджер                                                
Виктор Юдин   Павел Попов      83900    9300      0              14800
Данил Сидоров Илья Сергеев     38000       0   6700                  0
Женя Сидин    Илья Сергеев     42000       0      0               7000
Сергей Ильин  Илья Сергеев     84300       0  37600              21200
Сергей Минин  Павел Попов      43900       0  46900               2300

8. Вероятно полезно было бы рассматривать эффективность деятельности наших менеджеров не только по стоимости продаж, но и по их количеству. Добавьте в параметр values переменную  "Количество"

In [42]:
data_pivot = data_pd.pivot_table(index=['Контакт', 'Менеджер'], columns='Продукт', values=['Цена', 'Количество'],
                                  aggfunc='sum', fill_value=0)
data_pivot

Количество                                  \
Продукт                     Компьютер Монитор Софт Тех. сопровождение   
Контакт       Менеджер                                                  
Виктор Юдин   Павел Попов           6       4    0                  1   
Данил Сидоров Илья Сергеев          4       0    1                  0   
Женя Сидин    Илья Сергеев          4       0    0                  2   
Сергей Ильин  Илья Сергеев          9       0    2                  2   
Сергей Минин  Павел Попов           8       0    4                  2   

                                Цена                                    
Продукт                    Компьютер Монитор   Софт Тех. сопровождение  
Контакт       Менеджер                                                  
Виктор Юдин   Павел Попов      83900    9300      0              14800  
Данил Сидоров Илья Сергеев     38000       0   6700                  0  
Женя Сидин    Илья Сергеев     42000       0      0               7000  
Сергей Ильин  Илья Сергеев     84300       0  37600              21200  
Сергей Минин  Павел Попов      43900       0  46900               2300

9. Выведите продажи только менеджера "Илья Сергеев":

In [45]:
data_pivot.columns

MultiIndex([('Количество',          'Компьютер'),
            ('Количество',            'Монитор'),
            ('Количество',               'Софт'),
            ('Количество', 'Тех. сопровождение'),
            (      'Цена',          'Компьютер'),
            (      'Цена',            'Монитор'),
            (      'Цена',               'Софт'),
            (      'Цена', 'Тех. сопровождение')],
           names=[None, 'Продукт'])

In [46]:
data_pivot.loc[data_pivot.index.get_level_values('Менеджер') == 'Илья Сергеев']

Количество                                  \
Продукт                     Компьютер Монитор Софт Тех. сопровождение   
Контакт       Менеджер                                                  
Данил Сидоров Илья Сергеев          4       0    1                  0   
Женя Сидин    Илья Сергеев          4       0    0                  2   
Сергей Ильин  Илья Сергеев          9       0    2                  2   

                                Цена                                    
Продукт                    Компьютер Монитор   Софт Тех. сопровождение  
Контакт       Менеджер                                                  
Данил Сидоров Илья Сергеев     38000       0   6700                  0  
Женя Сидин    Илья Сергеев     42000       0      0               7000  
Сергей Ильин  Илья Сергеев     84300       0  37600              21200